# Computation of Cavs vectors

---
Goal of the notebook: .

Inputs of the notebook:
-
Output of the notebook:
-
Takeaways: 
- .
- .

In [ ]:
# !pip install -r requirements.txt

In [ ]:
import sys
sys.path.append('../../run_experiments/')
sys.path.append('../../run_experiments/scripts')
sys.path.append('../../run_experiments/models')
sys.path.append('../../run_experiments/data')

import pandas as pd
import json

# import fonction for getting PLM and tokenizer
from models.utils import load_model_and_tokenizer



# 0. autoreload

In [ ]:
#code for autoreload script associated with jupyter notebook
%load_ext autoreload
%autoreload 2

# Cavs creation process

config file

In [ ]:
# import config
from load_config import load_config

model_name = 'bert-base-uncased'    # 'bert-base-uncased' ou 'deberta-large'
dataset    = 'movies'               # 'movies' / 'agnews' / 'dbpedia' / 'medical'/ 'ledgar'/ n24news
annotation = 'C3M'       # 'C3M' ou 'our_annotation' ou 'combined_annotation'
config = load_config(model_name, dataset)
config.annotation = annotation

## choisir le mode de calcul des cavs : changer le truc du fichier config pour enregristrer les fichiers dans le bon dossier
# !!!!!!!! important !!!!!!!
config.cavs_type = 'mean'
# !!!!!!!! important !!!!!!!

load data

In [ ]:
# import the data
from prepare_data import load_fc_prepare_data
prepare_data = load_fc_prepare_data(config.DATASET)
train_loader, test_loader, val_loader, train_df, val_df, test_df = prepare_data(config)

load backbone to compute activation

In [ ]:
if config.model_name == 'gemma':
    from models.BaselineModel_gemma import BaselineModel
else:
    from models.BaselineModel import BaselineModel
# import the PLM model and tokenizer and bottleneck layer
embedder_model, embedder_tokenizer, ModelXtoCtoY_layer, classifier = load_model_and_tokenizer(config, 
                                                                                              n_concepts = 4
                                                                                             )


In [ ]:
black_box_model = BaselineModel(embedder_model, classifier, None, None, None, config, save_path = config.SAVE_PATH)
black_box_model.load_model()


### charger les dataframes et dataloaders augmentés avec les concepts 

In [ ]:
from prepare_data import prepare_data_from_csv

In [ ]:
df_aug_train = prepare_data_from_csv(annotation = 'combined_annotation', config = config) # C3M our_annotation C3M combined_annotation

In [ ]:
from concepts_bank_utils import create_dataloader
notre_loader_train = create_dataloader(df_aug_train, embedder_tokenizer, config.max_len, config.batch_size)


In [ ]:
from concepts_bank_utils import create_dataloader
# charger les données de train (facultatif : validation et de test augmented)
df_aug_train = pd.read_csv(f"{config.SAVE_PATH_CONCEPTS}/df_with_topics_v4.csv")


In [ ]:
if config.model_name == 'gemma':
    from mean_cavs_creation_gemma import compute_cavs_mean_minus
else:
    from mean_cavs_creation import compute_cavs_mean_minus

In [ ]:
# Calcul les cavs
cavs = compute_cavs_mean_minus(
    df_aug= df_aug_train,
    embedder_model=black_box_model.embedder_model,
    embedder_tokenizer=embedder_tokenizer,
    baseline_model=black_box_model,
    config=config,
)


## calculer le TCAVS score à partir des moyennes des embedding

In [ ]:
concept_name_list = []

# Consolidation des concepts dans une colonne unique
for column in df_aug_train.columns:
    if 'dummy' not in column:
        continue
    concept_name = column.replace('dummy_', '')
    concept_name_list.append(concept_name)

print(len(concept_name_list))

In [ ]:

# IMPORTANT LIBRARY TO IMPORT
from TCAVS import TCAV
from TCAVS_utils import stratified_subset_dataloader

TCAVS_ranker = TCAV(
    concepts=concept_name_list,
    baseline_model=black_box_model,
    embedder_tokenizer = embedder_tokenizer,
    batch_size=config.batch_size,
    config=config,
    verbose=True
)    

TCAVS_ranker.load_cavs_from_file()

tcav_scores = TCAVS_ranker.calculate_tcav_scores(
    notre_loader_train,
     use_subset=True,
)

### moyenne pour toutes les classes et enregistrer en json

In [ ]:
from ranking_utils import rank_macro_concepts
import json
# # Rank the macro concepts based on TCAV scores
sorted_macro_concepts = rank_macro_concepts(tcav_scores)

file_path = f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs/{config.cavs_type}"

with open(f'{file_path}/sorted_macro_concepts.json', 'w') as f:
     json.dump(sorted_macro_concepts, fp=f, default=lambda x: x.tolist())

print('sorted macro concepts enregistré ici:', f'{file_path}/sorted_macro_concepts.json')